In [99]:
"""
Queries Uniprot database and retrieves protein data.
"""
import urllib.request
from urllib.error import HTTPError
from lxml import etree
import lxml.etree
import lxml.html
import json
import xmlschema
import lxml
import requests
from bs4 import BeautifulSoup
import html
import pandas as pd

In [100]:
def unroll(data, master_list, ec):
    if type(data) == dict:
        for key, value in data.items():
            if(key == 'ecNumber'):
                ec = True
            unroll(data[key], master_list, ec)  
    elif type(data) == list:
        for i in range(0,len(data)):
            unroll(data[i], master_list, ec)
    else:
        if(type(data) != int):
            if(ec == True):
                data = "EC " + data
            master_list.append(data)

def findNewIDforOldID(oldID):
    A = 0
    print('Looking for the oldID', oldID)
    source = requests.get('https://www.uniprot.org/uniprot/'+oldID+ '.html').text
    webpage = BeautifulSoup(source, 'html')
    main = webpage.find('main')
    altIDsSection = main.find_all('div', class_= 'entryEmpty')
    altIDs = []
    for each_ID in altIDsSection:
        links = each_ID.find_all('a', href=True)
        for link in links:
            altIDlink = link['href'][9:]   #removes prefix of link containing ID
            altIDs.append(altIDlink)
    
    #Remove non-protein IDs from altIDs
    endOfaltIDs = 0
    for index in range(0, len(altIDs)):
        if('replaces' in altIDs[index]):
            endOfaltIDs = index
            break
    realAltIDs = []
    for index in range(0, endOfaltIDs):
        realAltIDs.append(altIDs[index])
    print('Stopped looking for the old ID. Found ', realAltIDs)
    return(realAltIDs)

def findNewIDforOldID_SecondAttempt(oldID):
    #FIRST: Finding the link to the next page /html/body/main/div/p/a[1]
    source = requests.get('https://www.uniprot.org/uniprot/'+oldID+ '.html').text
    webpage = BeautifulSoup(source, 'html')
    altID_Link = 'https://www.uniprot.org'+webpage.find('main').find('div', class_= 'entryEmpty').find("p").find("a")['href']
    
    print('The protein ID will be looked for at this link: ', altID_Link)

    #SECOND: Using the link for the next page to find the existing ID: /html/body/main/div/div[3]/table/tbody/tr/td[4]/a[2
                                                                    #  /html/body/main/div/div[3]/div[4]/table/tbody/tr[1]/td[2]/div/div]
    source2 = requests.get(altID_Link).text
    webpage2 = BeautifulSoup(source2, 'html')
    main = webpage2.find('main',class_='uniparc') #.find('div', class_='content results').find('table', class_='grid')#.find('tbody')
    step1 = main.find('div',class_='main').find('div',class_='content results').find('table').find('tbody').find('tr').find_all('td')
    thePossibleIDs = step1[3].find_all('a')
    thePossibleIDs_List = []
    for ID_Link in thePossibleIDs:
        thePossibleIDs_List.append(ID_Link['href'][9:])
    #Finds the ID(s) out of the final options from the last section
    theRightIDs = []
    for index in range(0, len(thePossibleIDs_List)):
        if('version' not in thePossibleIDs_List[index]):
            theRightIDs.append(thePossibleIDs_List[index])
    return(theRightIDs)

def findURLxml(ID):
    unrolledSynList = []
    url = 'https://www.uniprot.org/uniprot/'+ID+'.xml'
    with urllib.request.urlopen(url) as r:                     
        raw_data = r.read().strip()
    tree = etree.fromstring(raw_data); 
    entry_dict = schema.to_dict(tree);
    content = entry_dict['entry'][0]
    try: species_name = content['organism']['name'][0]['$']
    except: species_name = ''; print('Error with species name for ', ID)
    unroll(content['protein'], unrolledSynList, False)
    return unrolledSynList, species_name

def findUniRefURL(ID):
    webpage = BeautifulSoup(requests.get('https://www.uniprot.org/uniref/?query='+ID+'&fil=identity:1.0&sort=score').text, 'html')
    try: syn = str(webpage.find('main', class_= 'uniref').find('div', class_="main-aside").find('div', class_="content results").find('table', class_="grid").find('tbody').find('tr').find_all('td')[3]).split('>')[1].split(': ')[1].split('<')[0]    
    except: syn = []
    try: species_name = str(webpage.find('main', class_='uniref').find('div', class_="main-aside").find('div', class_="content results").find('table').find('tbody').find('tr').find_all('td')[6]).split('>')[2].split('<')[0]     #/html/body/main/div/div[2]/table/tbody/tr/td[7]/a
    except: species_name = ''
    #print(species_name)
    #print(syn)
    return syn, species_name


def findURL(ID):
    synList = []; species_name = 'N/A'; stillMissIDs = []; foundidList = []; notfoundidList = []
    try: 
        synList, species_name = findURLxml(ID); print('1st Try')
        foundidList.append(ID)
    except: 
        print('Not found on 1st try')
        try: 
            missing = False; UniRef = False
            altIDs = findNewIDforOldID(ID) #To find synonyms. If found on 2nd try. There might be multiple new IDs
            # If this is a dead end and no alternate IDs are provided, try UniRef
            if(altIDs == [] or altIDs == '[]'):
                try: 
                    SYN, SN = findUniRefURL(ID)
                    SYN = SYN.split(' ')[0]
                    if(SYN != [] and SYN != '[]'): print('SYN found on UniRef ', SYN, 'in ', SN); UniRef = True; altIDs = [ID]
                except: 
                    print('Couldn\'t find on UniRef either')
            # IF UniProt provides alternates, use those IDs to get their synonyms/names
            for altID in altIDs:
                if(UniRef == False):
                    try: syns, sn = findURLxml(altID); 
                    except: missing = True               # ...unless 
                elif(UniRef == True):
                    syns, species_name = [SYN], SN
                for s in syns: synList.append(s)
                foundidList.append(altID)
                print('2nd try')
                print('New ID for ', ID, ' is ', altID, ' on 2nd try')
                if missing == True: stillMissIDs.append(altID)
        
        except: print('Not all found on 2nd try'); stillMissIDs.append(ID) #If there weren't any new IDs
        if(len(stillMissIDs) > 0): #If there are still IDs without an entry
            for ID in stillMissIDs:
                try: 
                    altIDs2 = findNewIDforOldID_SecondAttempt(ID); print('3rd try')
                    for altID2 in altIDs2:
                        try: 
                            syns, sn = findURLxml(altID2) 
                            for s in syns: synList.append(s)
                            foundidList.append(altID2)
                            print('New ID for ', ID, ' is ', altID, ' on 3rd try')
                        except(HTTPError, lxml.etree.XMLSyntaxError) as err: print('Missing:', altID2); notfoundidList.append(altID2)
                except(HTTPError, lxml.etree.XMLSyntaxError) as err: print('Missing:', altID2); notfoundidList.append(ID)
    for missingname in notfoundidList: print('No Entry Found For ', missingname)
    return notfoundidList, foundidList, synList, species_name

In [101]:
df_reactome = pd.read_csv('Mitochondrial Proteome 1135.csv')
df_reactome

,Pathway identifier,Pathway name,#Entities found,#Entities total,#Interactors found,#Interactors total,Entities ratio,Entities pValue,Entities FDR,#Reactions found,#Reactions total,Reactions ratio,Species identifier,Species name,Submitted entities found,Mapped entities,Submitted entities hit interactor,Interacts with,Found reaction identifiers
0,R-HSA-1268020,Mitochondrial protein import,65,69,2,15,0.004745,1.110000e-16,1.840000e-14,14,14,0.001036,9606,Homo sapiens,O75746;Q96H96;Q9Y584;P56277;Q9BVV7;O96008;Q9NX...,O75746;Q96H96;Q9Y584;P56277;Q9BVV7;O96008;Q9NX...,Q9Y5J7;P62072,Q9Y5J6,R-HSA-1299475;R-HSA-1268022;R-HSA-1299478;R-HS...
1,R-HSA-5389840,Mitochondrial translation elongation,87,94,2,43,0.006464,1.110000e-16,1.840000e-14,8,8,0.000592,9606,Homo sapiens,Q96A35;Q96DV4;Q96EL2;Q7Z2W9;Q96EL3;Q9P015;Q9H0...,Q96A35;Q96DV4;Q96EL2;Q7Z2W9;Q96EL3;Q9P015;Q9H0...,Q8IVH4;P22033,15996,R-HSA-5389842;R-HSA-5389857;R-HSA-5419268;R-HS...
2,R-HSA-5419276,Mitochondrial translation termination,87,94,2,45,0.006464,1.110000e-16,1.840000e-14,5,5,0.000370,9606,Homo sapiens,Q96A35;Q96DV4;Q96EL2;Q7Z2W9;Q96EL3;Q9P015;Q9H0...,Q96A35;Q96DV4;Q96EL2;Q7Z2W9;Q96EL3;Q9P015;Q9H0...,Q8IVH4;P22033,15996,R-HSA-5419264;R-HSA-5419281;R-HSA-5419271;R-HS...
3,R-HSA-5368287,Mitochondrial translation,93,102,8,132,0.007014,1.110000e-16,1.840000e-14,17,17,0.001258,9606,Homo sapiens,Q96A35;Q96DV4;P46199;Q96EL2;Q7Z2W9;Q96EL3;Q9P0...,Q96A35;Q96DV4;P46199;Q96EL2;Q7Z2W9;Q96EL3;Q9P0...,Q9BVS5;Q8IVH4;Q8NI60;P22033;Q07817;Q9BUP3;O148...,Q9H2K0;P46199;15996,R-HSA-5389842;R-HSA-5419281;R-HSA-5389841;R-HS...
4,R-HSA-5368286,Mitochondrial translation initiation,87,96,6,86,0.006602,1.110000e-16,1.840000e-14,4,4,0.000296,9606,Homo sapiens,Q96A35;Q96DV4;P46199;Q96EL2;Q7Z2W9;Q96EL3;Q9P0...,Q96A35;Q96DV4;P46199;Q96EL2;Q7Z2W9;Q96EL3;Q9P0...,Q9BVS5;Q8NI60;Q07817;Q9BUP3;O14880;P17152,Q9H2K0;P46199,R-HSA-5389841;R-HSA-5368279;R-HSA-5389849;R-HS...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1580,R-HSA-9635486,Infection with Mycobacterium tuberculosis,0,134,30,724,0.009215,1.000000e+00,1.000000e+00,8,73,0.005401,9606,Homo sapiens,NaN,NaN,Q9BSH4;Q9BXW6;Q969S9;Q08426;Q9H3L0;Q96E11;Q4G1...,P27361;P28482;16856;P49840;P51149;O14964;P1437...,R-HSA-9635667;R-HSA-9636304;R-HSA-9635424;R-HS...
1581,R-HSA-8986944,Transcriptional Regulation by MECP2,1,100,68,1162,0.006877,1.000000e+00,1.000000e+00,8,77,0.005697,9606,Homo sapiens,P17612,P17612,Q969Q5;Q02218;Q16891;P45880;Q96EY1;Q9NPL8;Q6DK...,Q05329;P60484;P08581;P42858;P18031;Q14353,R-HSA-8986937;R-HSA-9023592;R-HSA-9615571;R-HS...
1582,R-HSA-1236394,Signaling by ERBB4,0,82,14,636,0.005639,1.000000e+00,1.000000e+00,5,52,0.003847,9606,Homo sapiens,NaN,NaN,Q9Y3D2;P17612;Q9BYZ2;Q8IVH4;Q9H7Z7;Q92843;Q96B...,P14136;16618;P04271;P02649;15996,R-HSA-9612243;R-HSA-1250370;R-HSA-1253321;R-HS...
1583,R-HSA-157118,Signaling by NOTCH,0,258,30,751,0.017742,1.000000e+00,1.000000e+00,14,154,0.011394,9606,Homo sapiens,NaN,NaN,P49406;P49327;P49916;P57105;Q5T1C6;Q16822;O603...,P41743;P05412;P31749;O00308;P84022;P01106;P678...,R-HSA-9021357;R-HSA-2168136;R-HSA-9017797;R-HS...


In [102]:
num = []
pathway_id = []
pathway_name = []
interactor_row = []
interacts_row = []
uniprotless_pathway = []
uniprot_dict = {}
interactor_dict = {}
interact_dict = {}
x = 0
for i in range(0,len(df_reactome)):
    if(type(df_reactome['Submitted entities hit interactor'][i]) == str):
        interactor = df_reactome['Submitted entities hit interactor'][i].split(';')
        interacts = df_reactome['Interacts with'][i].split(';')
        num.append(x); x += 1
        for b in range(0,len(interacts)):
            for j in range(0,len(interactor)):
                interacts_row.append(interacts[b])
                interactor_row.append(interactor[j])
        interactor_dict[i] = interactor_row
        interact_dict[i] = interacts_row
    else:
        uniprotless_pathway.append(df_reactome['Submitted entities hit interactor'][i])
        #print(i, ': ', df_reactome['Pathway identifier'][i])
print('Interactorless Pathway: ', len(uniprotless_pathway), '/', len(df_reactome), '=', len(uniprotless_pathway)/len(df_reactome))
#df_reactome2 = pd.DataFrame({'num': num, 'Submitted entities hit interactor':interactor_dict, 'Interacts with':interacts_dict})
#df_reactome2

Interactorless Pathway:  171 / 1585 = 0.10788643533123028


In [106]:
A_HIC = []
B_IWC = []
C_n = []
n = 0
for i in range(0,len(df_reactome)):
    hit_interactor_cell = df_reactome['Submitted entities hit interactor'][i]
    interacts_with_cell = df_reactome['Interacts with'][i]
    if(type(hit_interactor_cell) == str and type(interacts_with_cell) == str):
        hit_interactor_cell = hit_interactor_cell.split(';')
        interacts_with_cell = interacts_with_cell.split(';')
        for j in range(0,len(interacts_with_cell)):
            for k in range(0,len(hit_interactor_cell)):       
                A_HIC.append(hit_interactor_cell[k])
                B_IWC.append(interacts_with_cell[j])
                C_n.append(n)
                n += 1
df_interact = pd.DataFrame({'#':C_n,'Protein':A_HIC,'Interacts':B_IWC})

In [109]:
df_interact

,#,Protein,Interacts
0,0,Q9Y5J7,Q9Y5J6
1,1,P62072,Q9Y5J6
2,2,Q8IVH4,15996
3,3,P22033,15996
4,4,Q8IVH4,15996
...,...,...,...
340711,340711,P48047,P50454
340712,340712,P36542,P50454
340713,340713,P07195,P50454
340714,340714,Q53R41,P50454


In [110]:
df_interact.to_csv('proteins2proteins.csv')

In [81]:
IUP_set = {df_interact['Interacts With'][0]}
for i in range(0,len(df_interact)):
    IUP_set.add(df_interact['Interacts With'][i])
interact_with = list(IUP_set)
print('Unique Interactor Proteins:', len(IUP_set))

Unique 'Interacts with' Proteins: 731


### Get Interactor Names

In [82]:
f = open('./../MitoProteins.txt', 'r')
MitoProteins = {}
for line in f:
    ID = line.split('|')[0]
    names = line.split('|')[1:]
    MitoProteins[ID] = list(names)
MitoProteins

{'P08574': ['cytochrome c1, heme protein, mitochondrial-like',
  'Cytochrome c-1',
  'cytochrome c1, heme protein, mitochondrial isoform X2',
  'Complex III subunit IV',
  'Ubiquinol-cytochrome-c reductase complex cytochrome c1 subunit',
  'Cytochrome heme',
  'Cytochrome c1, heme protein, mitochondrial isoform 1',
  'Complex III subunit 4',
  'CYC1',
  'Cytochrome c1, heme protein, mitochondrial',
  'cytochrome c1',
  'Cytochrome c domain-containing protein',
  'Cytochrome c1',
  'CY1 protein (Fragment)',
  'Cytochrome C1, Heme protein, mitochondrial',
  'MC3DN6',
  'UQCR4',
  'EC 7.1.1.8',
  'cytochrome c1, heme protein, mitochondrial',
  'Cytochrome b-c1 complex subunit 4',
  'Cytochrome c-1 (Predicted), isoform CRA_c',
  'CY1 protein',
  'UPI001B34BCDF UniRef100 entry',
  'cytochrome c1, heme protein, mitochondrial isoform X1',
  'CYC1 isoform 1',
  'CYC1 isoform 2',
  'cytochrome c-1',
  'complex III subunit IV',
  'ubiquinol-cytochrome-c reductase complex cytochrome c1 subunit',


In [83]:
format = '.xml'; schema = xmlschema.XMLSchema('https://www.uniprot.org/docs/uniprot.xsd')
InteractorDict = {}; initial = 0;i = initial
synList = []; notfound = []; notfoundDict = {}

for ID in interact_with[initial:]:
    if(ID in MitoProteins.keys()):
        synList = MitoProteins[ID]
        print("Interactor is a submitted proteins")
    else:
        notfoundIDs, foundIDs, synList, sn = findURL(ID)
    InteractorDict[ID] = synList

1st Try
1st Try
1st Try
Interactor is a submitted proteins
1st Try
1st Try
1st Try
1st Try
1st Try
Not found on 1st try
Looking for the oldID 37550
Stopped looking for the old ID. Found  []
SYN found on UniRef  UPI001B37550E in  
2nd try
New ID for  37550  is  37550  on 2nd try
Not found on 1st try
Looking for the oldID 17526
Stopped looking for the old ID. Found  []
SYN found on UniRef  hypothetical in  Streptomyces sp. PKU-EA00015
2nd try
New ID for  17526  is  17526  on 2nd try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
Interactor is a submitted proteins
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
Interactor is a submitted proteins
1st Try
1st Try
1st Try
1st Try
Interactor is a submitted proteins
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
Not found on 1st try
Looking for the oldID 15996
Stopped looking 

1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
Interactor is a submitted proteins
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
Interactor is a submitted proteins
1st Try
1st Try
1st Try
1st Try
Interactor is a submitted proteins
1st Try
1st Try
1st Try
Interactor is a submitted proteins
1st Try
1st Try
1st Try
Interactor is a submitted proteins
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
Interactor is a submitted proteins
1st Try
1st Try
1st Try
1st Try
Interactor is a submitted proteins
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
Interactor is a submitted proteins
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
Interactor is a submitted proteins
1st Try
1st Try
1st Try
Interactor is a submitted proteins
1st Try
Interactor is a submitted proteins
1st Try
1st Try
Interactor is a submitted proteins
1st Try
1st Try
Interactor is a submitted proteins
1st Try
1st Try
N

In [84]:
len(ProteinDict)

731

In [94]:
IUP_syns

[]

In [96]:
IUP_names = []
IUP_syns = []
try: print(df_interact['Interacts With'][0])
except: print(df_interact['ID'][0])

for i in range(0,len(df_interact)):
    try: P = df_interact['Interacts With'][i]
    except: P = df_interact['ID'][i]
    if(P in InteractorDict.keys()):
        try: 
            IUP_names.append(InteractorDict[P][0])
        except:
            IUP_names.append(P)
        try:
            IUP_syns.append(InteractorDict[P][1:])
        except:
            IUP_syns.append('_')

df_interact = df_interact.assign(Name = IUP_names, Synonyms = IUP_syns)
df_interact.rename(columns={'Submitted Protein':'protein','Interacts With':'ID','Name':'name'}, inplace = True)
df_interact

# 'ID' Protein maps to the 'protein' Protein. 'name' is of 'ID' protein

Q9Y5J6


,#,protein,ID,name,name,Synonyms
0,0,Q9Y5J7,Q9Y5J6,Tim10b,Tim10b,[Mitochondrial import inner membrane transloca...
1,1,P62072,Q9Y5J6,Tim10b,Tim10b,[Mitochondrial import inner membrane transloca...
2,2,Q8IVH4,15996,Protein,Protein,[]
3,3,P22033,15996,Protein,Protein,[]
4,4,Q8IVH4,15996,Protein,Protein,[]
...,...,...,...,...,...,...
340711,340711,P48047,P50454,Serpin H1,Serpin H1,"[47 kDa heat shock protein, Arsenic-transactiv..."
340712,340712,P36542,P50454,Serpin H1,Serpin H1,"[47 kDa heat shock protein, Arsenic-transactiv..."
340713,340713,P07195,P50454,Serpin H1,Serpin H1,"[47 kDa heat shock protein, Arsenic-transactiv..."
340714,340714,Q53R41,P50454,Serpin H1,Serpin H1,"[47 kDa heat shock protein, Arsenic-transactiv..."


In [98]:
df_interact.to_csv('data/int2proteins.csv')